Following Free Code Camps tutorial closely.

In [27]:
import ollama
import numpy as np
import torch

In [4]:
question = "What is a Kalman filter"
document = "Chapter 5: AR and Kalman Filters"

In [12]:
import tiktoken

def num_tokens_from_str(string: str, encoding_name: str) -> int:
    """Returns number of tokens in a text string"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return encoding.encode(string), num_tokens

In [5]:
num_tokens_from_str(question, "cl100k_base")

6

In [30]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = OllamaEmbeddings(model="llama3.2")

query_result = embeddings.embed_query(question)
document_result = embeddings.embed_query(document)

In [63]:
print(str(query_result)[:100])
print(len(query_result))

[-0.0073224027, -0.019477012, 0.017059486, -0.0046530142, 0.0021997404, -0.009726169, 0.008544929, -
3072


In [32]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product/(norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print(similarity)

0.6177236244940539


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Lecture 1 - Basic circuit theory.pdf")
loader2 = PyPDFLoader("Lecture 7- The MOS transistors.pdf")
docs = loader.load()
docs2 = loader2.load()

print(f"Total characters: {len(docs[2].page_content)}")
print(f"Total characters: {len(docs2[5].page_content)}")
print(len(docs2))
print(docs2[5].page_content[:500])

Total characters: 514
Total characters: 659
42
6
 Purely symmetric device: source 
and drain are interchangeable 
depending on the applied voltages;
 To differentiate between source 
and drain, use the following 
definitions:
　 For NMOS, the source terminal is 
biased at a lower potential than 
the drain;
　 For PMOS, the source terminal is 
biased at a higher potential than 
the drain terminal.
The MOS Terminals
S
G
D
p-type substrate
B
n+ n+
 The BODY (also referred to as Bulk) terminal is usually connected to a 
fixed potential:
　 For N


In [69]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_docs = docs + docs2

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300,
    chunk_overlap=50
)

#Make splits
splits = text_splitter.split_documents(all_docs)

In [81]:
vectorstore = InMemoryVectorStore.from_documents(splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

From RAG Tutorial this appears to end the embedding and indexing section, next video we will see retrieval

# Part3 - Retrieval

In this section code camp goes over the retrieval section

In [71]:
docsearch = retriever.get_relevant_documents("How does an NMOS Transistor work")

In [74]:
docsearch2 = retriever.get_relevant_documents("List the basic axioms of circuit theory")

In [75]:
docsearch2

[Document(id='035b7775-12da-4255-9a4e-4a759f165036', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 0, 'page_label': '1'}, page_content='Lecture 1\nBasic circuit theory')]

In [73]:
docsearch

[Document(id='00edd6b1-e227-4867-94fc-94f37a416237', metadata={'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': "D:20240405072534Z00'00'", 'author': 'Wen', 'moddate': "D:20250521165548Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 7- The MOS transistors [Compatibility Mode]', 'source': 'Lecture 7- The MOS transistors.pdf', 'total_pages': 42, 'page': 2, 'page_label': '3'}, page_content='3\n50nm transistor dimension\n1.2nm Gate \nOxide\nSD\n~2000x smaller than diameter of human hair\nIntel 50nm transistor in production')]

In [82]:
docsearch3 = retriever.get_relevant_documents("What is a linear dependent source")

print(docsearch3)

[Document(id='9569fbc7-9c7a-4230-a052-05fba9e268c0', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 19, 'page_label': '20'}, page_content='20\nLinear circuit elements and dependent sources'), Document(id='c8adf07b-a58c-4acf-ac19-aa1979e6d36b', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 11, 'page_label': '12'}, page_content='

In [86]:
print(docsearch3[0].page_content)

20
Linear circuit elements and dependent sources


# Part 4 - Generation

In [87]:
from langchain.prompts import PromptTemplate

template = """Answer the following question only using the following context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the following question only using the following context:\n{context}\n\nQuestion: {question}\n')

In [88]:
llm = OllamaLLM(model = "llama3.2")

In [89]:
chain = prompt | llm

In [94]:
print(chain.invoke({"context":all_docs, "question":"How does KCL workf"}))

It appears you didn't provide any context or question related to KCL (Kirchhoff's Current Law). Kirchhoff's Current Law states that the current entering a node is equal to the current leaving the node.

However, I can infer that you might be referring to the concept of Kirchhoff's Laws in the context of electrical engineering, specifically with regards to MOSFETs (Metal-Oxide-Semiconductor Field-Effect Transistors).

If that's the case, I'd be happy to provide an answer. However, based on the provided text, it seems like there is no direct information about how KCL works in relation to MOSFETs.

The text does mention some basic concepts related to MOSFETs as switches and amplifiers, but it doesn't delve into the specifics of Kirchhoff's Current Law or its application to MOSFETs.
